## Late Chunking 실습

In [ ]:
# %pip install einops -U transformers

모델 로드 시에 문제가 생기면 아래 코드를 실행해주세요

In [ ]:
# import shutil
# import os

# cache_path = os.path.expanduser("~/.cache/huggingface")
# if os.path.exists(cache_path):
#     shutil.rmtree(cache_path)
#     print("✅ Hugging Face 캐시가 삭제되었습니다.")
# else:
#     print("ℹ️ 캐시 폴더가 존재하지 않습니다.")

### 🔧 jina-embeddings-v2 모델로 late-embedding 해보기

In [ ]:
# Load model directly
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)

#### 📝 문장 기반 텍스트 청킹 함수 (Text Chunking by Sentences)
이 함수는 late chunking용 특수 텍스트 분할기로, 일반 스플리터와 달리:

1. 토큰 위치 추적: 각 문장 청크가 원본의 어떤 토큰에 해당하는지 정확히 매핑
2. 모델 토크나이저 통합: 특정 모델이 텍스트를 이해하는 방식과 일치하는 분할 제공
3. 정밀한 문장 경계: 마침표+공백 패턴으로 의미 단위 보존

In [ ]:
def chunk_by_sentences(input_text: str, tokenizer):
    """
    입력 텍스트를 문장 단위로 분할하는 함수입니다.
    
    Args:
        input_text: 분할할 텍스트
        tokenizer: 사용할 토크나이저 객체
    
    Returns:
        chunks: 문장 단위로 분할된 텍스트 리스트
        span_annotations: 각 문장의 토큰 위치 정보 [(시작_토큰_인덱스, 끝_토큰_인덱스), ...]
    
    Note:
        오프셋(offset)은 텍스트 내에서 각 토큰의 시작과 끝 위치를 나타내는 인덱스 정보입니다.
        예를 들어, (5, 8)이라는 오프셋은 원본 텍스트의 5번째 문자부터 8번째 문자까지가 해당 토큰이라는 의미입니다.
        이를 통해 토큰화 후에도 원본 텍스트와의 매핑 관계를 유지할 수 있습니다.
    """
    # 입력 텍스트를 토크나이저로 처리하여 텐서와 오프셋 매핑 정보 얻기
    inputs = tokenizer(input_text, return_tensors='pt', return_offsets_mapping=True)
    # 마침표(.)와 [SEP] 토큰의 ID 가져오기
    punctuation_mark_id = tokenizer.convert_tokens_to_ids('.')
    sep_id = tokenizer.convert_tokens_to_ids('[SEP]')
    # 토큰의 오프셋과 ID 정보 추출
    token_offsets = inputs['offset_mapping'][0]
    token_ids = inputs['input_ids'][0]
    
    # 문장 끝 위치 찾기 (마침표 다음에 공백이 있거나 [SEP] 토큰이 오는 경우)
    chunk_positions = [
        (i, int(start + 1))
        for i, (token_id, (start, end)) in enumerate(zip(token_ids, token_offsets))
        if token_id == punctuation_mark_id
        and (
            token_offsets[i + 1][0] - token_offsets[i][1] > 0  # 마침표 다음에 공백이 있는지 확인
            or token_ids[i + 1] == sep_id  # 또는 [SEP] 토큰이 다음에 오는지 확인
        )
    ]
    
    # 찾은 위치를 기반으로 문장 단위로 텍스트 분할
    chunks = [
        input_text[x[1] : y[1]]
        for x, y in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    
    # 각 청크의 토큰 위치 정보 저장
    span_annotations = [
        (x[0], y[0]) for (x, y) in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    
    return chunks, span_annotations

In [ ]:
input_text = (
    "Berlin is the capital and largest city of Germany, both by area and by population. "
    "Its more than 3.85 million inhabitants make it the European Union's most populous city, "
    "as measured by population within city limits. The city is also one of the states of Germany, "
    "and is the third smallest state in the country in terms of area."
)

chunks, span_annotations = chunk_by_sentences(input_text, tokenizer)

In [ ]:
print(chunks)
print(span_annotations)

In [ ]:
import torch
import numpy as np

def late_chunking(model_output: 'BatchEncoding', span_annotation: list, max_length=None):
    """
    모델 출력에서 특정 스팬의 임베딩을 추출하여 평균 풀링을 수행합니다.
    
    Args:
        model_output: 모델의 출력 (BatchEncoding 객체)
        span_annotation: 각 문장에 대한 스팬(문장의 시작과 끝) 정보 목록 [(start, end), ...]
        max_length: 최대 시퀀스 길이 제한 (선택적)
    
    Returns:
        outputs: 각 스팬에 대한 평균 임베딩 벡터의 리스트
    """
    token_embeddings = model_output[0]  # (1, seq_len, hidden) 형태의 토큰 임베딩
    outputs = []
    
    # 각 배치의 임베딩과 스팬 정보를 순회
    for embeddings, annotations in zip(token_embeddings, span_annotation):
        # max_length가 지정된 경우 스팬 범위 제한
        if max_length is not None:
            annotations = [
                (start, min(end, max_length - 1))
                for (start, end) in annotations if start < (max_length - 1)
            ]
        
        # 각 스팬에 대해 평균 임베딩 계산 (스팬의 길이가 1 이상인 경우만)
        pooled = [
            embeddings[start:end].sum(dim=0) / (end - start)
            for start, end in annotations if (end - start) >= 1
        ]
        
        # 계산된 임베딩을 CPU 메모리로 이동하고 넘파이 배열로 변환
        outputs.append([p.detach().cpu().numpy() for p in pooled])
    
    return outputs

In [ ]:
from sentence_transformers.util import cos_sim

# 전통적: 청크 단위로 임베딩
traditional_embeddings = model.encode(chunks)

# Late Chunking: 전체 텍스트 → pooling
inputs = tokenizer(input_text, return_tensors='pt')
with torch.no_grad():
    model_output = model(**inputs)

late_embeddings = late_chunking(model_output, [span_annotations])[0]

In [ ]:
len(late_embeddings)

In [ ]:
query = "Berlin"
query_vec = model.encode(query)

print(f"\n🔍 Query: '{query}'")

for chunk, emb_late, emb_trad in zip(chunks, late_embeddings, traditional_embeddings):
    sim_late = cos_sim(torch.tensor(query_vec), torch.tensor(emb_late)).item()
    sim_trad = cos_sim(torch.tensor(query_vec), torch.tensor(emb_trad)).item()
    print(f"\n📌 Chunk: {chunk.strip()}\n- LateChunking 유사도: {sim_late:.4f}\n- 전통 방식 유사도: {sim_trad:.4f}")


### 🧠 PDF 문서를 Late Chunking 해보기

In [ ]:
v3_model = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True)
v3_tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True)

#### 📄 STEP 1. PDF 문서 로딩 및 텍스트 통합

In [ ]:
from langchain.document_loaders import PyPDFLoader

pdf_path = "./data/국가별 공공부문 AI 도입 및 활용 전략.pdf"  # ← 사용자의 PDF 경로
loader = PyPDFLoader(pdf_path)
pages = loader.load()
full_text = "\n".join([p.page_content for p in pages])

#### ✂️ STEP 2. 긴 문서 토큰 기준 슬라이스 (모델 입력 초과 방지)

✅ 모델 입력 제한 (8192 tokens) 기준으로 슬라이스 분할

In [ ]:
doc = Document(page_content=full_text)
splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    v3_tokenizer,
    chunk_size=8192
)
split_docs = splitter.split_documents([doc])
text_chunks = [d.page_content for d in split_docs]

#### 🧠 STEP 3. 문장 단위로 분할한 뒤, 임베딩 생성

In [ ]:
# 2. 각 청크에서 문장 단위 분할 및 임베딩 처리
all_sentences = []
all_embeddings = []

for chunk_text in text_chunks:
    # 각 청크를 문장 단위로 분할
    sentences, span_annotations = chunk_by_sentences(chunk_text, v3_tokenizer)
    
    # 현재 청크에 대한 임베딩 생성
    inputs = v3_tokenizer(chunk_text, return_tensors='pt')
    with torch.no_grad():
        model_output = v3_model(**inputs)
    
    # Late Chunking으로 문장별 임베딩 추출
    chunk_embeddings = late_chunking(model_output, [span_annotations])[0]
    
    # 결과 저장
    all_sentences.extend(sentences)
    all_embeddings.extend(chunk_embeddings)

# 이제 all_sentences와 all_embeddings에 모든 문장과 해당 임베딩이 저장됨
print(f"총 문장 수: {len(all_sentences)}")

#### 📌 STEP 4. 결과 확인

In [ ]:
# 임베딩 생성 및 유사도 검색
query = "영국 중앙 디지털데이터청(CDDO)의 역할과 그 전략적 파트너 기관은 누구이며, 함께 수립한 전략의 주요 목표는 무엇인가?"
query_vec = v3_model.encode(query, task="retrieval.query")

print(f"\n🔍 Query: '{query}'")

# 청크와 유사도 점수를 저장할 리스트
results = []

for chunk, emb_late in zip(all_sentences, all_embeddings):
    sim_late = cos_sim(torch.tensor(query_vec), torch.tensor(emb_late)).item()
    results.append({"chunk": chunk.strip(), "similarity": sim_late})

# 유사도 기준으로 내림차순 정렬
results = sorted(results, key=lambda x: x["similarity"], reverse=True)

# 가장 연관성 높은 3개 청크만 출력
print("\n🔝 가장 연관성 높은 3개 청크:")
for i, result in enumerate(results[:5]):
    print(f"\n📌 Chunk {i+1}: {result['chunk']}\n- 유사도: {result['similarity']:.4f}")
